In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.ml.stat import Correlation
from pyspark.sql.types import *
from datetime import datetime, timedelta
from pyspark.sql import Window
import time

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
547,application_1657150860135_0548,pyspark,idle,Link,Link,tungn,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%HTML
<script src="require.js"></script>

In [3]:
utils = ["utilization_all", "utilization_tx", "utilization_rx_in_bss", "utilization_rx_other_bss", "utilization_unknown_wifi", "utilization_non_wifi"]
clients = ["med_clients", "max_clients", "avg_clients", "min_clients"]
test_cols = ["bcn_per_wlan", "noise_floor", "tx_phy_err", "mac_stats_tx_phy_err", "tx_errors", "rx_errors"]
mean_stds = ["uptime"]
main_cols_ids = ["site_id", "org_id", "ap_id", "band"]
main_cols = ["org_name", "site_name", "ap_name", "band"]
ids = ["site_id", "org_id", "ap_id"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_org = spark.read.parquet("s3://mist-secorapp-production/dimension/org").select(F.col("id").alias("org_id"),F.col("name").alias("org_name")).persist()

df_name = spark.read.parquet("s3://mist-secorapp-production/dimension/site/site.parquet").select(F.col("id").alias("site_id"),F.col("name").alias("site_name"),F.col("org_id").alias("org_id"))\
    .join(df_org,["org_id"]).select("org_name","site_name","org_id","site_id").persist()

df_device_name = spark.read.parquet("s3://mist-secorapp-production/dimension/device").select(F.col("mac").alias("ap_id"),F.col("name").alias("ap_name"),"site_id","model","type").persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# the roundTo unit is minutes
def roundTime(dt_in=None, roundTo=60):
    if not dt_in:
            return None
    if isinstance(dt_in,str):
        try:
            dt = datetime.strptime(dt_in[0:19].replace("T"," "), '%Y-%m-%d %H:%M:%S')
        except Exception as e:
            print(e)
            return None
    elif not isinstance(dt_in,datetime):
        return None
    else:
        dt = dt_in
    seconds = (dt- dt.min).seconds
    roundTo = roundTo*60
    rounding = (seconds+roundTo/2) // roundTo * roundTo
    return dt + timedelta(0,rounding-seconds,-dt.microsecond)
def bcn_per_wlan(bcn, num_wlans, model=""):
    bcn = bcn if bcn != None else -1.0
    bcn_high = 700.0
    if model.find("AP41") > -1:
        bcn_norm = bcn if bcn < bcn_high else bcn_high
    else:
        bcn_norm = bcn / num_wlans if num_wlans > 0 else 0
    bcn_norm = bcn_norm if bcn_norm < bcn_high else bcn_high
    return float(bcn_norm)

bcn_per_wlan_norm = F.udf(bcn_per_wlan, FloatType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def read_data(dt, hr):
    df = spark.read.parquet("s3://mist-secorapp-production/ap-stats-analytics/ap-stats-analytics-production/dt={dt}/hr={hr}".format(dt=dt,hr=hr))    
    df = df.withColumn("radios", F.explode("radios"))\
                .select(*ids, "model", "radios.*", F.to_timestamp(F.col("when")/1000000).alias("timestamp"), "uptime")\
                .withColumn("num_wlans", F.size("wlans"))\
                .withColumn("bcn_per_wlan", bcn_per_wlan_norm(F.col("interrupt_stats_tx_bcn_succ"), F.col("num_wlans"), F.col("model")))\
                .where(F.col("bandwidth") != 0)\
                .filter((F.col("num_wlans") > 0) & (F.col("radios.radio_missing")==False) & (F.col("uptime") > 24*60*60))
    
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def filtering_data(df):
    first_cols = ["channel", "bandwidth", "re_init", "re_init_throttle", "tx_failed", "tx_retried", "tx_pkts", "rx_dups", "rx_pkts"] + test_cols + utils + ["wlans"]
    roundTimeUDF = F.udf(roundTime,TimestampType())
    df_filter = df.withColumn("time",roundTimeUDF("timestamp"))\
                .withColumn("time", F.from_utc_timestamp('time', 'PST'))\
                .groupby(*ids, "band", "time", "model")\
                .agg(*[F.max(col).alias(col) for col in first_cols],
                *[F.mean(col).alias("avg_" + col) for col in mean_stds], 
                *[F.stddev(col).alias("std_" + col) for col in mean_stds], 
                F.mean("num_clients").alias("avg_clients"), F.max("num_clients").alias("max_clients"), F.min("num_clients").alias("min_clients"),
                F.percentile_approx("num_clients", 0.5).alias("med_clients"),
                F.stddev("num_clients").alias("std_clients")).withColumn("ap_id", F.regexp_replace("ap_id", "-", ""))

    return df_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# join everything in here
def change_id_to_names(df, df_name=df_name, df_device_name=df_device_name, df_org=df_org):
    df_get_names = df.join(df_device_name, ["ap_id", "site_id"]).join(df_name, ["org_id", "site_id"]).distinct()
    return df_get_names

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# dt= "2022-06-08"
# dt= "2022-07-29"
dt= "2022-08-05"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# hrs = ["0" + str(i) for i in range(10)] + ["1" + str(i) for i in range(10)] + ["2" + str(i) for i in range(4)]
hrs = ["0" + str(i) for i in range(3, 10)]
for hr in hrs:
    df_read = read_data(dt, hr)
    df_filter = filtering_data(df_read)
    df_filter.write.mode("overwrite").parquet("s3://mist-data-science-dev/tung/edge/all/getnames/dt={dt}/hr={hr}".format(dt=dt,hr=hr))
    # df_filter.write.mode("overwrite").parquet("s3://mist-data-science-dev/tung/edge/all/getnames/dt={dt}/hr={hr}".format(dt=dt,hr=hr))
    del df_read, df_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# hrs = ["2" + str(i) for i in range(4)]
# # rewrite hr = 15, 17 -> 24
# for hr in hrs:
#     df_read = read_data(dt, hr)
#     df_filter = filtering_data(df_read)
#     df_filter.write.mode("overwrite").parquet("s3://mist-data-science-dev/tung/edge/all/getnames/dt={dt}/hr={hr}".format(dt=dt,hr=hr))
#     # df_filter.write.mode("overwrite").parquet("s3://mist-data-science-dev/tung/edge/all/getnames/dt={dt}/hr={hr}".format(dt=dt,hr=hr))
#     del df_read, df_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# df_read = read_data(dt, "*")
# df_filter = filtering_data(df_read)
# df_get_names = df_filter.withColumn("hr", F.hour("time"))
# # df_get_names = change_id_to_names(df_filter)
# del df_read, df_filter

In [ ]:
df_get_names = spark.read.parquet("s3://mist-data-science-dev/tung/edge/all/getnames/dt={dt}/hr=*".format(dt=dt))

# Add analysis of the correlation table

In [ ]:
groupby = ["zero_client", "bcn_per_wlan_drop", "noise_floor_high", "utilization_non_wifi_high", "tx_phy_err_high", "tx_error_high", "rx_error_high"]           

df_count = df_get_names.withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan")<500.0)\
        .withColumn("noise_floor_high", F.col("noise_floor")>-60.0) \
        .withColumn("zero_client", F.col("avg_clients")== 0.0)\
        .withColumn("tx_phy_err_high", F.col("tx_phy_err")>1.0) \
        .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi")>0.20)\
        .withColumn("tx_error_high", (F.col("tx_failed") + F.col("tx_retried"))/ F.col("tx_pkts") > 0.1) \
        .withColumn("rx_error_high", (F.col("rx_dups") + F.col("rx_errors"))/ F.col("rx_pkts") > 0.1)

In [ ]:
df_corr1 = df_count.groupby("band", *groupby).count().sort("count")

In [ ]:
# df_corr1.show(df_corr1.count(), truncate = False)

# Edge detection report - All Clients

In [ ]:
import time 
start = time.time()
m_win = 60*2
m_slide = 60
thre_clients = 2
detects = main_cols_ids + ["time"]
prevs = ["clients"] + ids
w1 = Window.partitionBy("org_id", "site_id", "ap_id", "band").orderBy("org_id", "site_id", "ap_id", "band", F.desc("time"))
w2 = F.window("time", windowDuration = f"{m_win} minutes", slideDuration = f"{m_slide} minutes")

In [ ]:
df_get_names = df_get_names.withColumn("clients", F.col("max_clients"))

In [ ]:
# df_long_0 = df_get_names.select("*", *[F.lag(c, offset=-1).over(w1).alias("prev_" + c) for c in prevs], *[F.lag(c, offset=-2).over(w1).alias("prev_2_" + c) for c in prevs])\
#                             .filter((F.col("prev_org_id") == F.col("prev_2_org_id")) & (F.col("org_id") == F.col("prev_org_id")))\
#                             .filter((F.col("prev_ap_id") == F.col("prev_2_ap_id")) & (F.col("ap_id") == F.col("prev_ap_id")))\
#                             .filter((F.col("prev_site_id") == F.col("prev_2_site_id")) & (F.col("site_id") == F.col("prev_site_id")))

df_long_0 = df_get_names.select("*", *[F.lag(c, offset=-2).over(w1).alias("prev_" + c) for c in prevs], *[F.lag(c, offset=-4).over(w1).alias("prev_2_" + c) for c in prevs])\
                            .filter((F.col("prev_org_id") == F.col("prev_2_org_id")) & (F.col("org_id") == F.col("prev_org_id")))\
                            .filter((F.col("prev_ap_id") == F.col("prev_2_ap_id")) & (F.col("ap_id") == F.col("prev_ap_id")))\
                            .filter((F.col("prev_site_id") == F.col("prev_2_site_id")) & (F.col("site_id") == F.col("prev_site_id")))

checks = detects + ["bcn_per_wlan", "noise_floor", "tx_phy_err", "mac_stats_tx_phy_err"]

df_edges = df_long_0.select(*checks, "clients", *["prev_" + c for c in prevs], *["prev_2_" + c for c in prevs])\
            .sort(*ids, "band", "time").groupby(*main_cols_ids, w2)\
            .agg(*[F.max(col).alias(col) for col in ["bcn_per_wlan", "noise_floor", "tx_phy_err"]],
                 F.avg("clients").alias("avg_clients"),
                 F.max("clients").alias("max_clients"),
                 F.count("clients").alias("count_clients"),
                 F.first("time").alias("time"),
                 F.first("prev_2_clients").alias("prev_2_clients"),
                 F.first("prev_clients").alias("prev_clients"),)\
            .filter(F.col("window.start") == F.col("time"))\
            .filter(F.col("count_clients") >= m_win/m_slide)\
            .filter((F.col("max_clients") == 0) & (F.col("avg_clients") == 0))\
            .filter((F.col("prev_clients") > 0) & (F.col("prev_2_clients") > 0))\
            .filter((F.col("prev_clients") > thre_clients) & (F.col("prev_2_clients") > thre_clients))\
#             .filter((F.col("bcn_per_wlan")<500.0) | (F.col("noise_floor")>-60.0) | (F.col("tx_phy_err")>1.0))

In [ ]:
site_id = '8cb91905-1843-4574-b7de-d2d0ae353cab'

In [ ]:
# df_edges.withColumn("hr", F.hour("time")).repartition("hr").write.partitionBy("hr").mode("overwrite").parquet("s3://mist-data-science-dev/tung/edge/all/edges/dt={dt}".format(dt=dt))

In [ ]:
# df_edges = spark.read.parquet("s3://mist-data-science-dev/tung/edge/all/edges/dt={dt}/hr=*".format(dt=dt))

In [ ]:
df_edges = change_id_to_names(df_edges)
df_edges = df_edges.select(*main_cols, "time", "ap_id")

In [ ]:
count_edges = df_edges.count()
print(count_edges)

In [ ]:
df_tesla = df_edges.filter(F.col("org_name") == "Tesla")
count = df_tesla.count()

In [ ]:
count

In [ ]:
df_tesla.show(count+10, truncate = False)

In [ ]:
df_tesla.filter((F.col("ap_id") == "a8537d00ea2e") & (F.col("band") == 5)).count()

### Total time of finding APs with edges

In [ ]:
end = time.time()
print(end - start)

In [ ]:
collections = df_tesla.select("ap_id", "band", "time").distinct().collect()

In [ ]:
### groupby = ["zero_client", "bcn_per_wlan_drop", "noise_floor_high", "utilization_non_wifi_high", "tx_phy_err_high", "tx_error_high", "rx_error_high", "num_wlans"]           
df_aps = None
df_timeframes = None
cond_1 = None
cond_2 = None

for i, ap in enumerate(collections):
    ap_name = ap[0]
    band_rad = ap[1]
    d_time = ap[2]
    statement_1 = (F.col("ap_id") == ap_name) & (F.col("band") == band_rad)
    statement_2 = (F.col("time")>= F.lit(d_time)) & (F.col("time") <= F.lit(d_time) + F.expr("INTERVAL 120 minutes"))
    cond_1 |= statement_1
    if i == 0:
        cond_2 = statement_2
    else:
        cond_2 |= statement_2

df_aps = df_get_names.filter(cond_1).sort("ap_id", "band", "time")
# df_timeframes = df_aps.filter(cond_2)
# df_timeframes.join(df_edges, *main_cols_ids).withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan")<500.0)\
#     .withColumn("noise_floor_high", F.col("noise_floor")>-60.0) \
#     .withColumn("tx_phy_err_high", F.col("tx_phy_err")>1.0) \
#     .withColumn("zero_client", F.col("avg_clients")== 0.0) \
#     .withColumn("num_wlans", F.size("wlans")) \
#     .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi")>0.30)\
#     .withColumn("tx_error_high", (F.col("tx_failed") + F.col("tx_retried"))/ F.col("tx_pkts") >0.1) \
#     .withColumn("rx_error_high", (F.col("rx_dups") + F.col("rx_errors"))/ F.col("rx_pkts")>0.1).groupby(*groupby).count().show(truncate = False)

In [ ]:
df_edges

In [ ]:
df_get_names

In [ ]:
groupby = ["zero_client", "bcn_per_wlan_drop", "noise_floor_high", "utilization_non_wifi_high", "tx_phy_err_high", "tx_error_high", "rx_error_high", "num_wlans"]           

df_frames = df_get_names.join(df_edges, ["ap_id", "band", "time"]).
    .withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan")<500.0)\
    .withColumn("noise_floor_high", F.col("noise_floor")>-60.0) \
    .withColumn("tx_phy_err_high", F.col("tx_phy_err")>1.0) \
    .withColumn("zero_client", F.col("clients")== 0.0) \
    .withColumn("num_wlans", F.size("wlans")) \
    .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi")>0.30)\
    .withColumn("tx_error_high", (F.col("tx_failed") + F.col("tx_retried"))/ F.col("tx_pkts") >0.1) \
    .withColumn("rx_error_high", (F.col("rx_dups") + F.col("rx_errors"))/ F.col("rx_pkts")>0.1)

In [ ]:
# df_frames = df_get_names.alias("f1").join(df_edges.alias("f2"), 
#         (F.col("f1.ap_id") == F.col("f2.ap_id")) & (F.col("f1.band") == F.col("f2.band")) & 
#         (F.col("f1.time")>= F.lit("f2.time")) & (F.col("f1.time") <= F.lit("f2.time") + F.expr("INTERVAL 120 minutes")), "inner")
#     .withColumn("bcn_per_wlan_drop", F.col("bcn_per_wlan")<500.0)\
#     .withColumn("noise_floor_high", F.col("noise_floor")>-60.0) \
#     .withColumn("tx_phy_err_high", F.col("tx_phy_err")>1.0) \
#     .withColumn("zero_client", F.col("clients")== 0.0) \
#     .withColumn("num_wlans", F.size("wlans")) \
#     .withColumn("utilization_non_wifi_high", F.col("utilization_non_wifi")>0.30)\
#     .withColumn("tx_error_high", (F.col("tx_failed") + F.col("tx_retried"))/ F.col("tx_pkts") >0.1) \
#     .withColumn("rx_error_high", (F.col("rx_dups") + F.col("rx_errors"))/ F.col("rx_pkts")>0.1).groupby(*groupby).count()

In [ ]:
df_frames.groupby(*groupby).count()\
        .sort(F.desc("count")).show(fcount, truncate = False)

# .filter((F.col("tx_phy_err_high") == True) | (F.col("bcn_per_wlan_drop") == True) | (F.col("noise_floor_high") == True))\

In [ ]:
df_frames.groupby(*groupby).count()\
        .filter((F.col("tx_phy_err_high") == True) | (F.col("bcn_per_wlan_drop") == True) | (F.col("noise_floor_high") == True))\
        .sort(F.desc("count")).show(fcount, truncate = False)

In [ ]:
d_time = "2022-06-08 07:00:00"
cond_2 = (F.col("time")>= F.lit(d_time)) & (F.col("time") <= F.lit(d_time) + F.expr("INTERVAL 120 minutes"))
df_timeframes = df_long_0.filter((F.col("ap_id") == "a8537d00ea2e") & (F.col("band") == 5))\
            .filter(cond_2).select("ap_id", "band", "time", "clients", "prev_clients", "prev_2_clients")

In [ ]:
df_timeframes.show(truncate = False)

In [ ]:
df_aps.select("site_id","org_id","ap_id","band").distinct().show(truncate = False)

In [ ]:
%%spark -o df_aps

In [ ]:
%%local
import plotly.express as px
import plotly.graph_objects as go

f_cols = ["clients"] + ["bcn_per_wlan", "noise_floor", "utilization_non_wifi", "tx_phy_err", "mac_stats_tx_phy_err", "tx_errors", "rx_errors"]

for i, column in enumerate(f_cols):
    fig = px.line(df_aps, x= "time" , y= column, color= "ap_id")
    fig.update_xaxes(showticklabels=True)
    fig.update_layout(title = f"{column} metric of the edge AP in 24 hours at Tesla")
    fig.show()